In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


Mounted at /content/drive


In [ ]:

%cd /content/drive/Shareddrives/CIS\ 519\ Final\ Project

/content/drive/Shareddrives/CIS 519 Final Project


In [ ]:
%ls
# 'Reviews.csv' should be listed here

'CIS 519 Project.ipynb'       'Project Milestone 2.gdoc'
 cleaned_reviews.csv          'Project Milestone 3.docx'
 fine-tuned-bert               res/
 James_CIS_519_Project.ipynb   Reviews.csv
 logs4/                       'Taha CIS 519 Project'
'Project 1 Milestone.gdoc'    'Time-Shift CIS 519 Project'


In [ ]:
import pandas as pd
import random
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

random.seed(42)
!pip install datasets

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 79.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 27.3 MB/s eta 0:00:00


In [ ]:
USE_CLEANED = True

In [ ]:
# we need to use these parameters to read in some funky text
if USE_CLEANED:
  df = pd.read_csv('cleaned_reviews.csv', engine='python', on_bad_lines='skip')
else:
  df = pd.read_csv('Reviews.csv', engine='python', on_bad_lines='skip')
  df['sentiment'] = df.apply(lambda x: 1 if x['Score'] > 3 else 0, axis=1)
  df = df.sample(frac=0.2).reset_index(drop=True)
  def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', ' ', text)
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    lemma = WordNetLemmatizer()
    result = [i for i in tokens if not i in stop_words]
    lemmatized_result = []
    for word in result:
      lemmatized_result.append(lemma.lemmatize(word))

    return " ".join(lemmatized_result)
    df['cleaned_text'] = df.apply(lambda x: preprocess_text(x['Text']), axis=1)

In [ ]:
df = df.dropna()

In [ ]:
df

,Unnamed: 0,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,sentiment,cleaned_text
0,0,119814,B0009YUEG2,A2F09EWKV3MTO2,StillWaters,4,4,5,1303430400,This is all I ever use anymore,I love this litter because...<br />1. It compl...,1,love litter br 1 completely eliminates urine o...
1,1,290509,B004777F3M,A2PDNBEIQYRCT2,lmac60,0,0,4,1309996800,Good Tea!,Great flavor. Just enough spice. A love Chai...,1,great flavor enough spice love chai tea latte ...
2,2,489194,B002N2R7WM,A1CHGDG3NUDA5R,"C. Seggelin ""Plastered Dragon""",0,0,1,1350086400,Tasteless Fat For Your Popcorn! Just Use Butter.,"Y'know, you'd think with the words ""Buttery Fl...",0,know think word buttery flavor product name pr...
3,3,65205,B001D0GV5O,A3GIKK6DQEKEQK,CHERYL A. CATANZANO,0,0,1,1296777600,Very bland,I bought this tea because I like Timothy's Cof...,0,bought tea like timothy coffee product figured...
4,4,526585,B007PA30TG,A2XQSJG8YR6ED2,Cecilia f.,0,0,5,1303084800,great brew,i've been using these k-cups for months now. ...,1,using k cup month breakfast blend perfect peop...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
113686,113686,16241,B007TJGZ54,A1PLASEQUSPDPM,Chris,0,0,5,1339718400,Price is Right,Love the Green Mountain Coffee; tastes great i...,1,love green mountain coffee taste great keurig ...
113687,113687,511398,B001E6J07I,A233H2J16V56W8,"Claxon,Bill",0,0,5,1237507200,Tasty Carrs cracker,A super glamourized cheez-it without the trans...,1,super glamourized cheez without trans fat chol...
113688,113688,211139,B000X67P2C,A3G0H0GKJ0722I,C. Smith,10,13,1,1277164800,Did not like...,I had heard so many good things about using th...,0,heard many good thing using place wheat white ...
113689,113689,520860,B0009XSXZM,A3FTHCRY3EZNA7,categ3893,1,1,5,1333843200,8 month Golden Retriever is OBSESSED.,My adolescent golden retriever just loves thes...,1,adolescent golden retriever love soon see hold...


In [ ]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.0 MB/s eta 0:00:00


In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification, DistilBertTokenizerFast
from transformers import InputExample, InputFeatures
from transformers import AutoTokenizer



In [ ]:
import torch

if torch.cuda.is_available():       
    device = torch.device("cuda:0")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla T4


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, log_loss, confusion_matrix

def get_metrics(predictions, actual):
  accuracy = accuracy_score(actual, predictions)
  f1 = f1_score(actual, predictions)
  cross_entropy = log_loss(actual, predictions)
  print(f"Accuracy: {accuracy:.2f}")
  print(f"F1 Score: {f1:.2f}")
  print(f"Cross-Entropy: {cross_entropy:.2f}")

In [ ]:
get_metrics(predictions, actual)

Accuracy: 0.65
F1 Score: 0.73
Cross-Entropy: 12.72


In [ ]:
def shift_negative_data(test_data):
  negative_data = test_data[test_data['sentiment'] == 0]
  negative_count = len(negative_data)
  positive_data = test_data[test_data['sentiment'] == 1]
  positive_sample = positive_data.sample(n=negative_count)
  return positive_sample.append(negative_data)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
tuning_df = df.sample(frac=0.1)

In [ ]:
train_df, test_df = train_test_split(tuning_df, test_size=0.2, random_state=42)

In [ ]:
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

In [ ]:
from transformers import DistilBertTokenizerFast


In [ ]:
train_df['sentiment'].describe()

count    3637.000000
mean        0.780588
std         0.413905
min         0.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: sentiment, dtype: float64

In [ ]:
from torch.utils.data import DataLoader
from transformers import DistilBertForSequenceClassification, AdamW, AutoTokenizer, AutoModelForSequenceClassification
import torch.optim as optim
import torch

In [ ]:
import numpy as np

In [ ]:
torch.cuda.is_available()

True

In [ ]:
from tqdm import tqdm

In [ ]:
from datasets import load_metric
 
def compute_metrics(logits, labels):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")
  
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}


def validate(model, val_loader):       
    model.eval() # set to eval mode to avoid batchnorm
    with torch.no_grad(): # avoid calculating gradients
        correct, total = 0, 0
        all_logits = None
        all_labels = None

        for data in tqdm(val_loader):
            input_ids = data['input_ids'].to(device)
            attention_mask = data['attention_mask'].to(device)
            labels = data['labels'].numpy()
            logits = model(input_ids=input_ids, attention_mask=attention_mask)["logits"].cpu().numpy()
            if all_labels is None:
              all_labels = labels
              all_logits = logits
            else:
              all_logits = np.concatenate((all_logits, logits))
              all_labels = np.concatenate((all_labels, labels))
        print(compute_metrics(all_logits, all_labels))

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')



In [ ]:
model_name = "siebert/sentiment-roberta-large-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
bert = AutoModelForSequenceClassification.from_pretrained(model_name)


NameError: ignored

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
count_parameters(bert)

355361794

In [ ]:
train_encodings = tokenizer(train_df['Text'].tolist(), truncation=True, padding=True, return_tensors='pt')
val_encodings = tokenizer(val_df['Text'].tolist(), truncation=True, padding=True, return_tensors='pt')
test_encodings = tokenizer(test_df['Text'].tolist(), truncation=True, padding=True, return_tensors='pt')
test_shift_df = shift_negative_data(test_df)
test_shift_encodings = tokenizer(test_shift_df['Text'].tolist(), truncation=True, padding=True, return_tensors='pt')

<ipython-input-15-4a862ef7d6af>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return positive_sample.append(negative_data)


In [ ]:
import torch
import torch.nn as nn
class AmazonDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = AmazonDataset(train_encodings, train_df['sentiment'].tolist())
val_dataset = AmazonDataset(val_encodings, val_df['sentiment'].tolist())
test_dataset = AmazonDataset(test_encodings, test_df['sentiment'].tolist())
test_shift_dataset = AmazonDataset(test_shift_encodings, test_shift_df['sentiment'].tolist())

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)
test_shift_loader = DataLoader(test_shift_dataset, batch_size=16, shuffle=True)

In [ ]:
validate(bert, test_loader)

  0%|          | 0/143 [00:00<?, ?it/s]<ipython-input-32-c891fd644c8f>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 143/143 [03:36<00:00,  1.51s/it]
<ipython-input-17-970661fcdf20>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")


{'accuracy': 0.934476693051891, 'f1': 0.9583682592903046}


In [ ]:
validate(bert, test_shift_loader)

  0%|          | 0/65 [00:00<?, ?it/s]<ipython-input-35-ef3e5d0fedeb>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 65/65 [01:35<00:00,  1.47s/it]
<ipython-input-20-970661fcdf20>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")


{'accuracy': 0.9083820662768031, 'f1': 0.9143897996357012}


In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
from sklearn.naive_bayes import GaussianNB


In [ ]:
# train for logistic regression
with torch.no_grad(): # avoid calculating gradients
    correct, total = 0, 0
    all_logits = None
    all_labels = None

    for data in tqdm(train_loader):
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        labels = data['labels'].numpy()
        logits = bert(input_ids=input_ids, attention_mask=attention_mask)["logits"].cpu().numpy()
        if all_labels is None:
          all_labels = labels
          all_logits = logits
        else:
          all_logits = np.concatenate((all_logits, logits))
          all_labels = np.concatenate((all_labels, labels))

  0%|          | 0/455 [00:00<?, ?it/s]<ipython-input-35-ef3e5d0fedeb>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 455/455 [11:25<00:00,  1.51s/it]


In [ ]:
with torch.no_grad(): # avoid calculating gradients
    correct, total = 0, 0
    test_logits = None
    test_labels = None

    for data in tqdm(test_loader):
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        labels = data['labels'].numpy()
        logits = bert(input_ids=input_ids, attention_mask=attention_mask)["logits"].cpu().numpy()
        if test_logits is None:
          test_logits = logits
          test_labels = labels
        else:
          test_logits = np.concatenate((test_logits, logits))
          test_labels = np.concatenate((test_labels, labels))

  0%|          | 0/143 [00:00<?, ?it/s]<ipython-input-35-ef3e5d0fedeb>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 143/143 [03:27<00:00,  1.45s/it]


In [ ]:
with torch.no_grad(): # avoid calculating gradients
    correct, total = 0, 0
    test_shift_logits = None
    test_shift_labels = None

    for data in tqdm(test_shift_loader):
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        labels = data['labels'].numpy()
        logits = bert(input_ids=input_ids, attention_mask=attention_mask)["logits"].cpu().numpy()
        if test_shift_logits is None:
          test_shift_logits = logits
          test_shift_labels = labels
        else:
          test_shift_logits = np.concatenate((test_shift_logits, logits))
          test_shift_labels = np.concatenate((test_shift_labels, labels))

  0%|          | 0/65 [00:00<?, ?it/s]<ipython-input-35-ef3e5d0fedeb>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 65/65 [01:33<00:00,  1.44s/it]


In [ ]:
lr = LogisticRegression(penalty='elasticnet', l1_ratio=0.5, solver='saga')
lr.fit(all_logits, all_labels)

LogisticRegression(l1_ratio=0.5, penalty='elasticnet', solver='saga')

In [ ]:
lr_preds = lr.predict(test_logits)
get_metrics(lr_preds, test_labels)

Accuracy: 0.95
F1 Score: 0.97
Cross-Entropy: 1.97


In [ ]:
lr_shift_preds = lr.predict(test_shift_logits)
get_metrics(lr_shift_preds, test_shift_labels)

Accuracy: 0.90
F1 Score: 0.90
Cross-Entropy: 3.76


In [ ]:
nbc = GaussianNB()
nbc.fit(all_logits, all_labels)

GaussianNB()

In [ ]:
nbc_preds = nbc.predict(test_logits)
get_metrics(nbc_preds, test_labels)

Accuracy: 0.95
F1 Score: 0.97
Cross-Entropy: 1.87


In [ ]:
nbc_shift_preds = nbc.predict(test_shift_logits)
get_metrics(nbc_shift_preds, test_shift_labels)

Accuracy: 0.91
F1 Score: 0.91
Cross-Entropy: 3.30


In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier()
dtree.fit(all_logits, all_labels)
dtree_preds = dtree.predict(test_logits)
get_metrics(dtree_preds, test_labels)

Accuracy: 0.91
F1 Score: 0.94
Cross-Entropy: 3.38


In [ ]:
import torch
import torch.nn as nn


In [ ]:
torch.cuda.empty_cache()

optimiz
for epoch in range(3):
  running_loss = 0.0
  for i,data in enumerate(tqdm(train_loader)):
    optimizer.zero_grad()
    input_ids = data['input_ids'].to(device)
    attention_mask = data['attention_mask'].to(device)
    labels = data['labels'].to(device)
    outputs = bert(input_ids, attention_mask=attention_mask)
    loss = outputs[0]
    # loss.requires_grad = True
    loss.backward()
    optimizer.step()
    running_loss += loss.item()  
  print('[%d] epoch loss: %.3f' % (epoch + 1, running_loss / len(train_loader) * 16))
  running_loss = 0.0

  validate(bert, val_loader)
  bert.train()
bert.eval()

  0%|          | 0/228 [00:03<?, ?it/s]


OutOfMemoryError: ignored

In [ ]:
from transformers import Trainer,TrainingArguments


In [ ]:
training_args = TrainingArguments(
    output_dir='./res',          # output directory
    evaluation_strategy="steps",
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=10,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs4',            # directory for storing logs
    #logging_steps=10,
    load_best_model_at_end=True,
)


In [ ]:
trainer = Trainer(
    model=bert,# the instantiated 🤗 Transformers model to be trained
    args=training_args, # training arguments, defined above
    train_dataset=train_dataset,# training dataset
    eval_dataset=val_dataset , # evaluation dataset
    compute_metrics=compute_metrics,
)
trainer.train()


<ipython-input-53-c891fd644c8f>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


OutOfMemoryError: ignored

In [ ]:
train_df.columns

Index(['Unnamed: 0', 'Id', 'ProductId', 'UserId', 'ProfileName',
       'HelpfulnessNumerator', 'HelpfulnessDenominator', 'Score', 'Time',
       'Summary', 'Text', 'sentiment', 'cleaned_text'],
      dtype='object')

In [ ]:
sum_train_encodings = tokenizer(train_df['Summary'].tolist(), truncation=True, padding=True, return_tensors='pt')
sum_val_encodings = tokenizer(val_df['Summary'].tolist(), truncation=True, padding=True, return_tensors='pt')
sum_test_encodings = tokenizer(test_df['Summary'].tolist(), truncation=True, padding=True, return_tensors='pt')
sum_test_shift_encodings = tokenizer(test_shift_df['Summary'].tolist(), truncation=True, padding=True, return_tensors='pt')

In [ ]:
train_dataset = AmazonDataset(sum_train_encodings, train_df['sentiment'].tolist())
val_dataset = AmazonDataset(sum_val_encodings, val_df['sentiment'].tolist())
test_dataset = AmazonDataset(sum_test_encodings, test_df['sentiment'].tolist())
test_shift_dataset = AmazonDataset(sum_test_shift_encodings, test_shift_df['sentiment'].tolist())

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)
test_shift_loader = DataLoader(test_shift_dataset, batch_size=16, shuffle=True)

In [ ]:
validate(bert, test_loader)

  0%|          | 0/143 [00:00<?, ?it/s]<ipython-input-35-ef3e5d0fedeb>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 143/143 [00:21<00:00,  6.72it/s]


{'accuracy': 0.8918205804749341, 'f1': 0.9295532646048109}


In [ ]:
validate(bert, test_shift_loader)

  0%|          | 0/65 [00:00<?, ?it/s]<ipython-input-35-ef3e5d0fedeb>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 65/65 [00:11<00:00,  5.77it/s]


{'accuracy': 0.854775828460039, 'f1': 0.8636779505946934}


In [ ]:
with torch.no_grad(): # avoid calculating gradients
    correct, total = 0, 0
    all_logits = None
    all_labels = None

    for data in tqdm(train_loader):
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        labels = data['labels'].numpy()
        logits = bert(input_ids=input_ids, attention_mask=attention_mask)["logits"].cpu().numpy()
        if all_labels is None:
          all_labels = labels
          all_logits = logits
        else:
          all_logits = np.concatenate((all_logits, logits))
          all_labels = np.concatenate((all_labels, labels))

  0%|          | 0/455 [00:00<?, ?it/s]<ipython-input-35-ef3e5d0fedeb>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 455/455 [01:06<00:00,  6.82it/s]


In [ ]:
with torch.no_grad(): # avoid calculating gradients
    correct, total = 0, 0
    test_logits = None
    test_labels = None

    for data in tqdm(test_loader):
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        labels = data['labels'].numpy()
        logits = bert(input_ids=input_ids, attention_mask=attention_mask)["logits"].cpu().numpy()
        if test_logits is None:
          test_logits = logits
          test_labels = labels
        else:
          test_logits = np.concatenate((test_logits, logits))
          test_labels = np.concatenate((test_labels, labels))

  0%|          | 0/143 [00:00<?, ?it/s]<ipython-input-35-ef3e5d0fedeb>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 143/143 [00:20<00:00,  6.83it/s]


In [ ]:
with torch.no_grad(): # avoid calculating gradients
    correct, total = 0, 0
    test_shift_logits = None
    test_shift_labels = None

    for data in tqdm(test_shift_loader):
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        labels = data['labels'].numpy()
        logits = bert(input_ids=input_ids, attention_mask=attention_mask)["logits"].cpu().numpy()
        if test_shift_logits is None:
          test_shift_logits = logits
          test_shift_labels = labels
        else:
          test_shift_logits = np.concatenate((test_shift_logits, logits))
          test_shift_labels = np.concatenate((test_shift_labels, labels))

  0%|          | 0/65 [00:00<?, ?it/s]<ipython-input-35-ef3e5d0fedeb>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 65/65 [00:08<00:00,  8.12it/s]


In [ ]:
lr = LogisticRegression(penalty='elasticnet', l1_ratio=0.5, solver='saga')
lr.fit(all_logits, all_labels)

LogisticRegression(l1_ratio=0.5, penalty='elasticnet', solver='saga')

In [ ]:
lr_preds = lr.predict(test_logits)
get_metrics(lr_preds, test_labels)

Accuracy: 0.89
F1 Score: 0.93
Cross-Entropy: 3.79


In [ ]:
lr_preds = lr.predict(test_shift_logits)
get_metrics(lr_preds, test_shift_labels)

Accuracy: 0.81
F1 Score: 0.83
Cross-Entropy: 6.89


In [ ]:
rows_to_keep = (train_df.shape[0] // 16) * 16
rows_to_drop = train_df.shape[0] - rows_to_keep
train_df = train_df.drop(train_df.tail(rows_to_drop).index)

In [ ]:
train_tokens = tokenizer(train_df['Text'].tolist(), truncation=True, padding=True, return_tensors='pt')

In [ ]:
train_tokens

{'input_ids': tensor([[    0, 41552,   102,  ...,     1,     1,     1],
        [    0,   347,  9898,  ...,     1,     1,     1],
        [    0,   100,  1166,  ...,     1,     1,     1],
        ...,
        [    0,   100,    21,  ...,     1,     1,     1],
        [    0,   100,  2740,  ...,     1,     1,     1],
        [    0,  1106,    47,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [ ]:
train_dataset = AmazonDataset(train_tokens, train_df['sentiment'].tolist())
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

In [ ]:
rows_to_keep = (val_df.shape[0] // 16) * 16
rows_to_drop = val_df.shape[0] - rows_to_keep
val_df = val_df.drop(val_df.tail(rows_to_drop).index)

In [ ]:
val_tokens = tokenizer(val_df['Text'].tolist(), truncation=True, padding=True, return_tensors='pt')
val_dataset = AmazonDataset(val_tokens, val_df['sentiment'].tolist())
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=True)

In [ ]:
vocab_size = np.max(train_tokens['input_ids'].numpy()) + 1

In [ ]:
class SentimentalLSTM(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):    
        """
        Initialize the model by setting up the layers
        """
        super().__init__()
        self.output_size=output_size
        self.n_layers=n_layers
        self.hidden_dim=hidden_dim
        
        #Embedding and LSTM layers
        self.embedding=nn.Embedding(vocab_size, embedding_dim)
        self.lstm=nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        
        #dropout layer
        self.dropout=nn.Dropout(0.3)
        
        #Linear and sigmoid layer
        self.fc1=nn.Linear(hidden_dim, 64)
        self.fc2=nn.Linear(64, 16)
        self.fc3=nn.Linear(16,output_size)
        self.sigmoid=nn.Sigmoid()
        
    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size=x.size()
        
        #Embadding and LSTM output
        embedd=self.embedding(x)
        lstm_out, hidden=self.lstm(embedd, hidden)
        
        #stack up the lstm output
        lstm_out=lstm_out.contiguous().view(-1, self.hidden_dim)
        
        #dropout and fully connected layers
        out=self.dropout(lstm_out)
        out=self.fc1(out)
        out=self.dropout(out)
        out=self.fc2(out)
        out=self.dropout(out)
        out=self.fc3(out)
        sig_out=self.sigmoid(out)
        
        sig_out=sig_out.view(batch_size, -1)
        sig_out=sig_out[:, -1]
        
        return sig_out, hidden
    
    def init_hidden(self, batch_size):
        """Initialize Hidden STATE"""
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
              weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
                
        return hidden


In [ ]:
lstm = SentimentalLSTM(vocab_size, 2, 512, 256, 2)

In [ ]:
lr=0.001

criterion = nn.BCELoss()
optimizer = optim.Adam(lstm.parameters(), lr=lr)

# check if CUDA is available

# training params

epochs = 3 # 3-4 is approx where I noticed the validation loss stop decreasing

counter = 0
print_every = 100
clip=5 # gradient clipping

# move model to GPU, if available
lstm.cuda()
batch_size=16
lstm.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = lstm.init_hidden(batch_size)

    # batch loop
    for data in tqdm(train_loader):
        counter += 1
        inputs=data['input_ids'].cuda()
        labels=data['labels'].cuda()
        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        optimizer.zero_grad()

        # get the output from the model
        output, h = lstm(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(lstm.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = lstm.init_hidden(batch_size)
            val_losses = []
            lstm.eval()
            for data in val_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                inputs, labels = data['input_ids'].cuda(), data['labels'].cuda()  
                output, val_h = lstm(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())
            lstm.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))


  0%|          | 0/454 [00:00<?, ?it/s]<ipython-input-137-ef3e5d0fedeb>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
  0%|          | 0/454 [00:00<?, ?it/s]


RuntimeError: ignored

In [ ]:
rows_to_keep = (test_df.shape[0] // 16) * 16
rows_to_drop = test_df.shape[0] - rows_to_keep
test_df = test_df.drop(test_df.tail(rows_to_drop).index)
test_tokens = tokenizer(test_df['Text'].tolist(), truncation=True, padding=True, return_tensors='pt')
test_dataset = AmazonDataset(test_tokens, test_df['sentiment'].tolist())
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

In [ ]:
lstm.eval()
batch_size=16
lstm.train()
# train for some number of epochs
predictions = None
actual = None
with torch.no_grad(): # avoid calculating gradients

      # initialize hidden state
  h = lstm.init_hidden(batch_size)

  # batch loop
  for data in tqdm(test_loader):
      counter += 1
      inputs=data['input_ids'].cuda()
      labels=data['labels'].cuda()
      # Creating new variables for the hidden state, otherwise
      # we'd backprop through the entire training history
      h = tuple([each.data for each in h])

      # zero accumulated gradients

      # get the output from the model
      output, h = lstm(inputs, h)
      output = output.cpu().numpy()
      labels = labels.cpu().numpy()
      if actual is None:
        actual = labels
        predictions = output
      else:
        actual = np.concatenate((actual, labels))
        predictions = np.concatenate((predictions, output))
      # calculate the loss and   perform backprop
      # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.


  0%|          | 0/142 [00:00<?, ?it/s]<ipython-input-137-ef3e5d0fedeb>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 142/142 [00:12<00:00, 11.62it/s]


In [ ]:
get_metrics(predictions, actual)

ValueError: ignored

In [ ]:
round_lam = lambda x: 1 if x >= 0.5 else 0

In [ ]:
preds = np.round(predictions).astype(int)

In [ ]:
get_metrics(preds, actual)

Accuracy: 0.78
F1 Score: 0.87
Cross-Entropy: 8.07


In [ ]:
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout_rate, 
                 pad_index):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_index)
        self.convs = nn.ModuleList([nn.Conv1d(embedding_dim, 
                                              n_filters, 
                                              filter_size) 
                                    for filter_size in filter_sizes])
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout_rate)
        
    def forward(self, ids):
        # ids = [batch size, seq len]
        embedded = self.dropout(self.embedding(ids))
        # embedded = [batch size, seq len, embedding dim]
        embedded = embedded.permute(0,2,1)
        # embedded = [batch size, embedding dim, seq len]
        conved = [torch.relu(conv(embedded)) for conv in self.convs]
        # conved_n = [batch size, n filters, seq len - filter_sizes[n] + 1]
        pooled = [conv.max(dim=-1).values for conv in conved]
        # pooled_n = [batch size, n filters]
        cat = self.dropout(torch.cat(pooled, dim=-1))
        # cat = [batch size, n filters * len(filter_sizes)]
        prediction = self.fc(cat)
        # prediction = [batch size, output dim]
        return prediction


In [ ]:
embedding_dim = 300
n_filters = 100
filter_sizes = [3,5,7]
output_dim = 2
dropout_rate = 0.25



In [ ]:
model

CNN(
  (embedding): Embedding(50018, 300, padding_idx=1)
  (convs): ModuleList(
    (0): Conv1d(300, 100, kernel_size=(3,), stride=(1,))
    (1): Conv1d(300, 100, kernel_size=(5,), stride=(1,))
    (2): Conv1d(300, 100, kernel_size=(7,), stride=(1,))
  )
  (fc): Linear(in_features=300, out_features=2, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)

In [ ]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = criterion.to(device)
model.train()
epoch_losses = []

i = 0
for epoch in range(10):
  for batch in tqdm(train_loader):
      ids = batch['input_ids'].to(device)
      label = batch['labels'].to(device)
      prediction = model(ids)
      loss = criterion(prediction, label)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      epoch_losses.append(loss.item())
      i += 1
  training_loss = np.mean(epoch_losses)
  epoch_losses = []
  model.eval()
  for batch in (val_loader):
      ids = batch['input_ids'].to(device)
      label = batch['labels'].to(device)
      prediction = model(ids)
      loss = criterion(prediction, label)
      epoch_losses.append(loss.item())
      i += 1
  print(f"Epoch {epoch}, Training Loss {training_loss}, Validation Loss {np.mean(epoch_losses)} ")
  model.train()
  epoch_losses = []



  0%|          | 0/454 [00:00<?, ?it/s]<ipython-input-137-ef3e5d0fedeb>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 454/454 [00:10<00:00, 45.31it/s]


Epoch 0, Training Loss 0.5549752439663537, Validation Loss 0.39515249404759534 


100%|██████████| 454/454 [00:09<00:00, 46.86it/s]


Epoch 1, Training Loss 0.390333431768785, Validation Loss 0.3876902919295615 


100%|██████████| 454/454 [00:09<00:00, 47.25it/s]


...

Epoch 7, Training Loss 0.1271694922953201, Validation Loss 0.8917034369990213 


100%|██████████| 454/454 [00:09<00:00, 46.85it/s]


Epoch 8, Training Loss 0.14709639758076157, Validation Loss 0.6364556069710432 


100%|██████████| 454/454 [00:09<00:00, 45.52it/s]


Epoch 9, Training Loss 0.0893486283361928, Validation Loss 0.7337993283104975 


In [ ]:
import torchtext


In [ ]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
def tokenize_data(example, tokenizer, max_length):
    tokens = tokenizer(example['Text'])[:max_length]
    return tokens
max_length = 256

train_df['tokens'] = train_df.apply(lambda x: tokenize_data(x, tokenizer, max_length), axis=1)
test_df['tokens'] = test_df.apply(lambda x: tokenize_data(x, tokenizer, max_length), axis=1)


In [ ]:
min_freq = 5
special_tokens = ['<unk>', '<pad>']

vocab = torchtext.vocab.build_vocab_from_iterator(train_df['tokens'],
                                                  min_freq=min_freq,
                                                  specials=special_tokens)


In [ ]:
unk_index = vocab['<unk>']
pad_index = vocab['<pad>']
vocab.set_default_index(unk_index)
def numericalize_data(example, vocab):
    ids = [vocab[token] for token in example['tokens']]
    return ids
train_df['ids'] = train_df.apply(lambda x: numericalize_data(x, vocab), axis=1)
test_df['ids'] = test_df.apply(lambda x: numericalize_data(x, vocab), axis=1)

In [ ]:
vectors = torchtext.vocab.FastText()
pretrained_embedding = vectors.get_vecs_by_tokens(vocab.get_itos())
model = CNN(vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout_rate, pad_index)

.vector_cache/wiki.en.vec: 6.60GB [01:17, 85.2MB/s]                           
100%|██████████| 2519370/2519370 [06:56<00:00, 6046.13it/s]


In [ ]:
def initialize_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight)
        nn.init.zeros_(m.bias)
    elif isinstance(m, nn.Conv1d):
        nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
        nn.init.zeros_(m.bias)
model.apply(initialize_weights)
model.embedding.weight.data = pretrained_embedding


In [ ]:
def collate(batch, pad_index):
    batch_ids = [i['ids'] for i in batch]
    batch_ids = nn.utils.rnn.pad_sequence(batch_ids, padding_value=pad_index, batch_first=True)
    batch_label = [i['sentiment'] for i in batch]
    batch_label = torch.stack(batch_label)
    batch = {'ids': batch_ids,
             'label': batch_label}
    return batch


In [ ]:
batch_size = 512
import functools
collate = functools.partial(collate, pad_index=pad_index)

train_dataset = AmazonDataset(train_df['ids'].tolist(), train_df['sentiment'].tolist())
train_dataloader = torch.utils.data.DataLoader(train_dataset, 
                                               batch_size=batch_size, 
                                               shuffle=True)

# valid_dataloader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size, collate_fn=collate)
test_dataloader = torch.utils.data.DataLoader(test_df, batch_size=batch_size, collate_fn=collate)


In [ ]:
model.train()
epoch_losses = []
epoch_accs = []

for batch in tqdm(train_dataloader):
    ids = batch['ids'].to(device)
    label = batch['label'].to(device)
    prediction = model(ids)
    loss = criterion(prediction, label)
    accuracy = get_accuracy(prediction, label)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    epoch_losses.append(loss.item())
    epoch_accs.append(accuracy.item())

return epoch_losses, epoch_accs


  0%|          | 0/15 [00:00<?, ?it/s]


AttributeError: ignored

In [ ]:
train_dataset